# Input Data, Variables, and Parameters

In [36]:
import numpy as np
import os
import pandas as pd
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()
from arcgis.geoanalytics import manage_data

from IPython.display import display, Markdown

In [37]:
dirWork         = os.getcwd()
dirInput        = os.path.join(dirWork, r'input'       )
dirIntermediate = os.path.join(dirWork, r'intermediate')
dirResults      = os.path.join(dirWork, r'results'     )
dirParams       = os.path.join(dirWork, r'params'      )
dirWebApp       = os.path.join(dirResults, r'webapp'   )

sStationCol     = 'STATION'
sDayCol         = 'DAY'
sDOWCol         = 'DOW'
sMonthCol       = 'MONTH'
sYearCol        = 'YEAR'
sYearGrpCol     = 'YEARGROUP'
sSeasonGrpCol   = 'SEASONGROUP'
sMonthADTCol    = 'MONTHADT'
sMonthDOWADTCol = 'MONTHDOWADT'
sYearADTCol     = 'YEARADT'
sStationGrpCol  = 'STATIONGROUP'
sDOWGrpCol      = 'DOWGROUP'
sDOWFieldName   = 'DOWFLDNM'
sSGFieldName    = 'SGFLDNM'


In [38]:
#read parameter tables
#csv: , parse_dates=['DATEFROM','DATETO']
prmDOWToDayGroups         = pd.read_csv(os.path.join(dirParams, r'dow_to_dow_groups.csv'              ))
prmMonthToSeasonGroups    = pd.read_csv(os.path.join(dirParams, r'month_to_season_groups.csv'         ))
prmYearToYearGroups       = pd.read_csv(os.path.join(dirParams, r'year_to_year_groups.csv'            ))
prmYearGroups             = pd.read_csv(os.path.join(dirParams, r'year_groups.csv'                    ))
prmGeographies            = pd.read_csv(os.path.join(dirParams, r'geographies.csv'                    ))
prmStationToStationGroups = pd.read_csv(os.path.join(dirParams, r'station_to_station_groups.csv'      ))
prmStationGroups          = pd.read_csv(os.path.join(dirParams, r'station_groups.csv'                 ))
prmDOWFactorsMaxMin       = pd.read_csv(os.path.join(dirParams, r'dow_factors_maxmin.csv'             ))

prmFieldNamesSeasonGroup  = pd.read_csv(os.path.join(dirParams, r'season_group_field_names.csv'       ))
prmFieldNamesDOWGroup     = pd.read_csv(os.path.join(dirParams, r'dow_group_field_names.csv'          ))
 
prmWebAppDOWGroups        = pd.read_csv(os.path.join(dirParams, r'dow_webapp.csv'                     ))
prmWebAppSeasonGroups     = pd.read_csv(os.path.join(dirParams, r'season_webapp.csv'                  ))
 
prmDerivedStationPortions = pd.read_csv(os.path.join(dirParams, r'station_groups_derived_portions.csv'))

In [39]:
#read processed data
dfMonthADT    = pd.read_csv(os.path.join(dirIntermediate, r'MonthADT.csv'))
dfMonthDOWADT = pd.read_csv(os.path.join(dirIntermediate, r'MonthDOWADT.csv'))
dfYearADT     = pd.read_csv(os.path.join(dirIntermediate, r'YearADT.csv'))

In [40]:
#processed outputs
csv_factors = os.path.join(dirIntermediate,'CCS_Factors.csv')

In [41]:
#json outputs
json_dowfactors        = os.path.join(dirWebApp, r'dowfactors.json'       )
json_stationgroups     = os.path.join(dirWebApp, r'stationgroups.json'    )
json_stationgroups_ccs = os.path.join(dirWebApp, r'stationgroups_ccs.json')
json_wdfactors         = os.path.join(dirWebApp, r'wdfactors.json'        )
json_yeargroups        = os.path.join(dirWebApp, r'yeargroups.json'       )
json_monthfactors      = os.path.join(dirWebApp, r'monthfactors.json'     )
json_seasonfactors     = os.path.join(dirWebApp, r'seasonfactors.json'    )

# Year/Season Group ADTs

In [42]:
#calculate year adt for each grouping
dfYearGroupADTs = pd.DataFrame.merge(dfYearADT,prmYearToYearGroups,on=(sYearCol))
dfYearGroupADTs = pd.DataFrame.merge(dfYearGroupADTs,prmStationToStationGroups,on=(sStationCol))
dfYearGroupADTs = dfYearGroupADTs.groupby([sStationGrpCol,sYearGrpCol],as_index=False).agg(AVGYEARADT=(sYearADTCol,'mean'))
dfYearGroupADTs

,STATIONGROUP,YEARGROUP,AVGYEARADT
0,301,2013,49525.395238
1,301,2013-2017,54698.437659
2,301,2014,51804.412302
3,301,2015,54572.420833
4,301,2015-2019,57627.999504
...,...,...,...
1671,WDK,2015-2019,24444.463542
1672,WDK,2016,23810.038690
1673,WDK,2018,25320.111706
1674,WDK,2019,25960.541071


# Season Factors

In [43]:
#calculate month adt for each grouping
dfSeasonGroupADTs = pd.DataFrame.merge(dfMonthADT,prmMonthToSeasonGroups,on=(sMonthCol))
dfSeasonGroupADTs = pd.DataFrame.merge(dfSeasonGroupADTs,prmYearToYearGroups,on=(sYearCol))
dfSeasonGroupADTs = pd.DataFrame.merge(dfSeasonGroupADTs,prmStationToStationGroups,on=(sStationCol))
dfSeasonGroupADTs = dfSeasonGroupADTs.groupby([sStationGrpCol,sSeasonGrpCol,sYearGrpCol],as_index=False).agg(AVGSEASONADT=(sMonthADTCol,'mean'))
dfSeasonGroupADTs

,STATIONGROUP,SEASONGROUP,YEARGROUP,AVGSEASONADT
0,301,M01-Jan,2013,44843.840476
1,301,M01-Jan,2013-2017,52119.840476
2,301,M01-Jan,2014,51059.159524
3,301,M01-Jan,2015,51268.092857
4,301,M01-Jan,2015-2019,55225.066071
...,...,...,...,...
28487,WDK,Year,2015-2019,24444.463542
28488,WDK,Year,2016,23810.038690
28489,WDK,Year,2018,25320.111706
28490,WDK,Year,2019,25960.541071


In [44]:
dfSeasonFactors = pd.DataFrame.merge(dfSeasonGroupADTs,dfYearGroupADTs,on=(sStationGrpCol,sYearGrpCol))
dfSeasonFactors['SEASONFACTOR'] = dfSeasonFactors['AVGSEASONADT'] / dfSeasonFactors['AVGYEARADT'] 
dfSeasonFactors

,STATIONGROUP,SEASONGROUP,YEARGROUP,AVGSEASONADT,AVGYEARADT,SEASONFACTOR
0,301,M01-Jan,2013,44843.840476,49525.395238,0.905472
1,301,M02-Feb,2013,45251.178571,49525.395238,0.913696
2,301,M03-Mar,2013,46548.835714,49525.395238,0.939898
3,301,M04-Apr,2013,42423.921429,49525.395238,0.856609
4,301,M05-May,2013,43828.209524,49525.395238,0.884964
...,...,...,...,...,...,...
28487,WDK,S01-Winter,All Years,21859.765397,23761.535000,0.919964
28488,WDK,S02-Spring,All Years,22015.717937,23761.535000,0.926528
28489,WDK,S03-Summer,All Years,27620.208413,23761.535000,1.162392
28490,WDK,S04-Fall,All Years,23550.448254,23761.535000,0.991116


In [45]:
#get field names
dfSeasonFactors = pd.DataFrame.merge(dfSeasonFactors, prmFieldNamesSeasonGroup, on=sSeasonGrpCol)
dfSeasonFactors = dfSeasonFactors.rename(columns={sSGFieldName:'FACTORNAME','SEASONFACTOR':'FACTOR'})
dfSeasonFactors = dfSeasonFactors[[sStationGrpCol,sSeasonGrpCol,sYearGrpCol,'FACTORNAME','FACTOR']]
dfSeasonFactors

,STATIONGROUP,SEASONGROUP,YEARGROUP,FACTORNAME,FACTOR
0,301,M01-Jan,2013,FAC_JAN,0.905472
1,301,M01-Jan,2013-2017,FAC_JAN,0.952858
2,301,M01-Jan,2014,FAC_JAN,0.985614
3,301,M01-Jan,2015,FAC_JAN,0.939451
4,301,M01-Jan,2015-2019,FAC_JAN,0.958303
...,...,...,...,...,...
28487,WDK,Year,2015-2019,FAC_YR,1.000000
28488,WDK,Year,2016,FAC_YR,1.000000
28489,WDK,Year,2018,FAC_YR,1.000000
28490,WDK,Year,2019,FAC_YR,1.000000


# Max Month Factor

In [46]:
prmMonthToSeasonGroups_justMonths = prmMonthToSeasonGroups[prmMonthToSeasonGroups['SGTYPE']=='Month']
prmMonthToSeasonGroups_justMonths

,MONTH,SEASONGROUP,SGTYPE
0,1,M01-Jan,Month
1,2,M02-Feb,Month
2,3,M03-Mar,Month
3,4,M04-Apr,Month
4,5,M05-May,Month
5,6,M06-Jun,Month
6,7,M07-Jul,Month
7,8,M08-Aug,Month
8,9,M09-Sep,Month
9,10,M10-Oct,Month


In [47]:
dfMaxMonthFactor = pd.DataFrame.merge(dfSeasonFactors,prmMonthToSeasonGroups_justMonths,on=sSeasonGrpCol)
dfMaxMonthFactorMax = dfMaxMonthFactor.groupby([sStationGrpCol,sYearGrpCol],as_index=False).agg(FAC_MAX=('FACTOR','max'))
dfMaxMonthFactorMax['FACTORNAME']='FAC_MAX'
dfMaxMonthFactorMax[sSeasonGrpCol]='Year'
dfMaxMonthFactorMax = dfMaxMonthFactorMax.rename(columns={'FAC_MAX':'FACTOR'})
dfMaxMonthFactorMax = dfMaxMonthFactorMax[[sStationGrpCol,sSeasonGrpCol,sYearGrpCol,'FACTORNAME','FACTOR']]
dfMaxMonthFactorMax

,STATIONGROUP,SEASONGROUP,YEARGROUP,FACTORNAME,FACTOR
0,301,Year,2013,FAC_MAX,1.247890
1,301,Year,2013-2017,FAC_MAX,1.185596
2,301,Year,2014,FAC_MAX,1.162994
3,301,Year,2015,FAC_MAX,1.184390
4,301,Year,2015-2019,FAC_MAX,1.180784
...,...,...,...,...,...
1671,WDK,Year,2015-2019,FAC_MAX,1.190121
1672,WDK,Year,2016,FAC_MAX,1.165954
1673,WDK,Year,2018,FAC_MAX,1.183243
1674,WDK,Year,2019,FAC_MAX,1.217706


In [48]:
dfMaxMonthFactorMo = pd.DataFrame.merge(dfSeasonFactors,dfMaxMonthFactorMax,left_on=(sStationGrpCol,sYearGrpCol,'FACTOR'),right_on=(sStationGrpCol,sYearGrpCol,'FACTOR'))
dfMaxMonthFactorMo = pd.DataFrame.merge(dfMaxMonthFactorMo,prmMonthToSeasonGroups_justMonths,left_on=(sSeasonGrpCol+'_x'),right_on=(sSeasonGrpCol))
dfMaxMonthFactorMo = dfMaxMonthFactorMo[[sStationGrpCol,sYearGrpCol,'MONTH']]
dfMaxMonthFactorMo = dfMaxMonthFactorMo.rename(columns={'MONTH':'FACTOR'})
dfMaxMonthFactorMo[sSeasonGrpCol] = 'Year'
dfMaxMonthFactorMo['FACTORNAME'] = 'FAC_MAXMO'
dfMaxMonthFactorMo = dfMaxMonthFactorMo[[sStationGrpCol,sSeasonGrpCol,sYearGrpCol,'FACTORNAME','FACTOR']]
dfMaxMonthFactorMo


,STATIONGROUP,SEASONGROUP,YEARGROUP,FACTORNAME,FACTOR
0,317,Year,2019,FAC_MAXMO,1
1,605,Year,2013,FAC_MAXMO,1
2,605,Year,2013-2017,FAC_MAXMO,1
3,605,Year,2014,FAC_MAXMO,1
4,605,Year,2015,FAC_MAXMO,1
...,...,...,...,...,...
1671,704,Year,2019,FAC_MAXMO,12
1672,704,Year,All Years,FAC_MAXMO,12
1673,SNV,Year,2018,FAC_MAXMO,12
1674,U70,Year,2018,FAC_MAXMO,12


# DOW Factors

In [49]:
dfMonthDOWGroupADTs = pd.DataFrame.merge(dfMonthDOWADT,prmDOWToDayGroups,on=(sDOWCol))
dfMonthDOWGroupADTs = dfMonthDOWGroupADTs.groupby([sStationCol,sYearCol,sMonthCol,sDOWGrpCol],as_index=False).agg(MONTHDOWGROUPADT=(sMonthDOWADTCol,'mean'))
dfMonthDOWGroupADTs

,STATION,YEAR,MONTH,DOWGROUP,MONTHDOWGROUPADT
0,301,2013,1,1-Weekday (Mo-Th),45192.783333
1,301,2013,1,2-Weekend (Fr-Su),44378.583333
2,301,2013,1,D1-Monday,41221.333333
3,301,2013,1,D2-Tuesday,44297.000000
4,301,2013,1,D3-Wednesday,47701.400000
...,...,...,...,...,...
75163,714,2019,12,D3-Wednesday,3636.666667
75164,714,2019,12,D4-Thursday,3517.750000
75165,714,2019,12,D5-Friday,3612.000000
75166,714,2019,12,D6-Saturday,2618.750000


In [50]:
#calculate month adt for each grouping
dfSeasonGroupDOWADTs = pd.DataFrame.merge(dfMonthDOWGroupADTs,prmMonthToSeasonGroups,on=(sMonthCol))
dfSeasonGroupDOWADTs = pd.DataFrame.merge(dfSeasonGroupDOWADTs,prmYearToYearGroups,on=(sYearCol))
dfSeasonGroupDOWADTs = pd.DataFrame.merge(dfSeasonGroupDOWADTs,prmStationToStationGroups,on=(sStationCol))
dfSeasonGroupDOWADTs = dfSeasonGroupDOWADTs.groupby([sStationGrpCol,sSeasonGrpCol,sYearGrpCol,sDOWGrpCol],as_index=False).agg(AVGMONTHDOWADT=('MONTHDOWGROUPADT','mean'))
dfSeasonGroupDOWADTs

,STATIONGROUP,SEASONGROUP,YEARGROUP,DOWGROUP,AVGMONTHDOWADT
0,301,M01-Jan,2013,1-Weekday (Mo-Th),45192.783333
1,301,M01-Jan,2013,2-Weekend (Fr-Su),44378.583333
2,301,M01-Jan,2013,D1-Monday,41221.333333
3,301,M01-Jan,2013,D2-Tuesday,44297.000000
4,301,M01-Jan,2013,D3-Wednesday,47701.400000
...,...,...,...,...,...
256423,WDK,Year,All Years,D3-Wednesday,24058.063611
256424,WDK,Year,All Years,D4-Thursday,25027.441111
256425,WDK,Year,All Years,D5-Friday,27784.625556
256426,WDK,Year,All Years,D6-Saturday,24347.585278


In [51]:
dfSeasonGroupDOWFactors = pd.DataFrame.merge(dfSeasonGroupDOWADTs,dfSeasonGroupADTs,on=(sStationGrpCol,sSeasonGrpCol,sYearGrpCol))
dfSeasonGroupDOWFactors['DOWFACTOR'] = dfSeasonGroupDOWFactors['AVGMONTHDOWADT'] / dfSeasonGroupDOWFactors['AVGSEASONADT']
dfSeasonGroupDOWFactors

,STATIONGROUP,SEASONGROUP,YEARGROUP,DOWGROUP,AVGMONTHDOWADT,AVGSEASONADT,DOWFACTOR
0,301,M01-Jan,2013,1-Weekday (Mo-Th),45192.783333,44843.840476,1.007781
1,301,M01-Jan,2013,2-Weekend (Fr-Su),44378.583333,44843.840476,0.989625
2,301,M01-Jan,2013,D1-Monday,41221.333333,44843.840476,0.919220
3,301,M01-Jan,2013,D2-Tuesday,44297.000000,44843.840476,0.987806
4,301,M01-Jan,2013,D3-Wednesday,47701.400000,44843.840476,1.063722
...,...,...,...,...,...,...,...
256423,WDK,Year,All Years,D3-Wednesday,24058.063611,23761.535000,1.012479
256424,WDK,Year,All Years,D4-Thursday,25027.441111,23761.535000,1.053275
256425,WDK,Year,All Years,D5-Friday,27784.625556,23761.535000,1.169311
256426,WDK,Year,All Years,D6-Saturday,24347.585278,23761.535000,1.024664


In [52]:
dfSeasonGroupDOWFactors_toCombine = pd.DataFrame.merge(dfSeasonGroupDOWFactors,prmFieldNamesDOWGroup,on='DOWGROUP')
dfSeasonGroupDOWFactors_toCombine = dfSeasonGroupDOWFactors_toCombine.rename(columns={sDOWFieldName:'FACTORNAME','DOWFACTOR':'FACTOR'})
dfSeasonGroupDOWFactors_toCombine = dfSeasonGroupDOWFactors_toCombine[[sStationGrpCol,sSeasonGrpCol,sYearGrpCol,'FACTORNAME','FACTOR']]
dfSeasonGroupDOWFactors_toCombine

,STATIONGROUP,SEASONGROUP,YEARGROUP,FACTORNAME,FACTOR
0,301,M01-Jan,2013,FAC_WDAVG,1.007781
1,301,M01-Jan,2013-2017,FAC_WDAVG,1.002032
2,301,M01-Jan,2014,FAC_WDAVG,0.984669
3,301,M01-Jan,2015,FAC_WDAVG,1.004514
4,301,M01-Jan,2015-2019,FAC_WDAVG,1.012129
...,...,...,...,...,...
256423,WDK,Year,2015-2019,FAC_SUN,0.766689
256424,WDK,Year,2016,FAC_SUN,0.765462
256425,WDK,Year,2018,FAC_SUN,0.764525
256426,WDK,Year,2019,FAC_SUN,0.760448


# Max and Min of DOWFactors

In [53]:
dfSeasonGroupDOWFactorsMax = pd.DataFrame.merge(dfSeasonGroupDOWFactors,prmDOWFactorsMaxMin,on='DOWGROUP')
dfSeasonGroupDOWFactorsMax = dfSeasonGroupDOWFactorsMax[dfSeasonGroupDOWFactorsMax['MAXMIN']=='max']
dfSeasonGroupDOWFactorsMax = dfSeasonGroupDOWFactorsMax.groupby([sStationGrpCol,sSeasonGrpCol,sYearGrpCol,'DOWMAXMINGROUP','MAXMIN'],as_index=False).agg(MINMAXDOWFACTOR=('DOWFACTOR','max'))
dfSeasonGroupDOWFactorsMax

,STATIONGROUP,SEASONGROUP,YEARGROUP,DOWMAXMINGROUP,MAXMIN,MINMAXDOWFACTOR
0,301,M01-Jan,2013,FAC_WEMAX,max,1.071602
1,301,M01-Jan,2013-2017,FAC_WEMAX,max,1.133720
2,301,M01-Jan,2014,FAC_WEMAX,max,1.135122
3,301,M01-Jan,2015,FAC_WEMAX,max,1.134632
4,301,M01-Jan,2015-2019,FAC_WEMAX,max,1.146998
...,...,...,...,...,...,...
28487,WDK,Year,2015-2019,FAC_WEMAX,max,1.168174
28488,WDK,Year,2016,FAC_WEMAX,max,1.168849
28489,WDK,Year,2018,FAC_WEMAX,max,1.161396
28490,WDK,Year,2019,FAC_WEMAX,max,1.163974


In [54]:
dfSeasonGroupDOWFactorsMin = pd.DataFrame.merge(dfSeasonGroupDOWFactors,prmDOWFactorsMaxMin,on='DOWGROUP')
dfSeasonGroupDOWFactorsMin = dfSeasonGroupDOWFactorsMin[dfSeasonGroupDOWFactorsMin['MAXMIN']=='min']
dfSeasonGroupDOWFactorsMin = dfSeasonGroupDOWFactorsMin.groupby([sStationGrpCol,sSeasonGrpCol,sYearGrpCol,'DOWMAXMINGROUP','MAXMIN'],as_index=False).agg(MINMAXDOWFACTOR=('DOWFACTOR','min'))
dfSeasonGroupDOWFactorsMin

,STATIONGROUP,SEASONGROUP,YEARGROUP,DOWMAXMINGROUP,MAXMIN,MINMAXDOWFACTOR


In [55]:
dfSeasonGroupDOWFactorsMinMax_toCombine = dfSeasonGroupDOWFactorsMax.append(dfSeasonGroupDOWFactorsMin, ignore_index=True)
dfSeasonGroupDOWFactorsMinMax_toCombine = dfSeasonGroupDOWFactorsMinMax_toCombine.rename(columns={'DOWMAXMINGROUP':'FACTORNAME','MINMAXDOWFACTOR':'FACTOR'})
dfSeasonGroupDOWFactorsMinMax_toCombine = dfSeasonGroupDOWFactorsMinMax_toCombine.drop(columns='MAXMIN')
dfSeasonGroupDOWFactorsMinMax_toCombine


,STATIONGROUP,SEASONGROUP,YEARGROUP,FACTORNAME,FACTOR
0,301,M01-Jan,2013,FAC_WEMAX,1.071602
1,301,M01-Jan,2013-2017,FAC_WEMAX,1.133720
2,301,M01-Jan,2014,FAC_WEMAX,1.135122
3,301,M01-Jan,2015,FAC_WEMAX,1.134632
4,301,M01-Jan,2015-2019,FAC_WEMAX,1.146998
...,...,...,...,...,...
28487,WDK,Year,2015-2019,FAC_WEMAX,1.168174
28488,WDK,Year,2016,FAC_WEMAX,1.168849
28489,WDK,Year,2018,FAC_WEMAX,1.161396
28490,WDK,Year,2019,FAC_WEMAX,1.163974


# Combine Factors into single table

In [56]:
display(dfSeasonFactors)
display(dfMaxMonthFactorMax)
display(dfMaxMonthFactorMo)
display(dfSeasonGroupDOWFactors_toCombine)
display(dfSeasonGroupDOWFactorsMinMax_toCombine)

,STATIONGROUP,SEASONGROUP,YEARGROUP,FACTORNAME,FACTOR
0,301,M01-Jan,2013,FAC_JAN,0.905472
1,301,M01-Jan,2013-2017,FAC_JAN,0.952858
2,301,M01-Jan,2014,FAC_JAN,0.985614
3,301,M01-Jan,2015,FAC_JAN,0.939451
4,301,M01-Jan,2015-2019,FAC_JAN,0.958303
...,...,...,...,...,...
28487,WDK,Year,2015-2019,FAC_YR,1.000000
28488,WDK,Year,2016,FAC_YR,1.000000
28489,WDK,Year,2018,FAC_YR,1.000000
28490,WDK,Year,2019,FAC_YR,1.000000


,STATIONGROUP,SEASONGROUP,YEARGROUP,FACTORNAME,FACTOR
0,301,Year,2013,FAC_MAX,1.247890
1,301,Year,2013-2017,FAC_MAX,1.185596
2,301,Year,2014,FAC_MAX,1.162994
3,301,Year,2015,FAC_MAX,1.184390
4,301,Year,2015-2019,FAC_MAX,1.180784
...,...,...,...,...,...
1671,WDK,Year,2015-2019,FAC_MAX,1.190121
1672,WDK,Year,2016,FAC_MAX,1.165954
1673,WDK,Year,2018,FAC_MAX,1.183243
1674,WDK,Year,2019,FAC_MAX,1.217706


,STATIONGROUP,SEASONGROUP,YEARGROUP,FACTORNAME,FACTOR
0,317,Year,2019,FAC_MAXMO,1
1,605,Year,2013,FAC_MAXMO,1
2,605,Year,2013-2017,FAC_MAXMO,1
3,605,Year,2014,FAC_MAXMO,1
4,605,Year,2015,FAC_MAXMO,1
...,...,...,...,...,...
1671,704,Year,2019,FAC_MAXMO,12
1672,704,Year,All Years,FAC_MAXMO,12
1673,SNV,Year,2018,FAC_MAXMO,12
1674,U70,Year,2018,FAC_MAXMO,12


,STATIONGROUP,SEASONGROUP,YEARGROUP,FACTORNAME,FACTOR
0,301,M01-Jan,2013,FAC_WDAVG,1.007781
1,301,M01-Jan,2013-2017,FAC_WDAVG,1.002032
2,301,M01-Jan,2014,FAC_WDAVG,0.984669
3,301,M01-Jan,2015,FAC_WDAVG,1.004514
4,301,M01-Jan,2015-2019,FAC_WDAVG,1.012129
...,...,...,...,...,...
256423,WDK,Year,2015-2019,FAC_SUN,0.766689
256424,WDK,Year,2016,FAC_SUN,0.765462
256425,WDK,Year,2018,FAC_SUN,0.764525
256426,WDK,Year,2019,FAC_SUN,0.760448


,STATIONGROUP,SEASONGROUP,YEARGROUP,FACTORNAME,FACTOR
0,301,M01-Jan,2013,FAC_WEMAX,1.071602
1,301,M01-Jan,2013-2017,FAC_WEMAX,1.133720
2,301,M01-Jan,2014,FAC_WEMAX,1.135122
3,301,M01-Jan,2015,FAC_WEMAX,1.134632
4,301,M01-Jan,2015-2019,FAC_WEMAX,1.146998
...,...,...,...,...,...
28487,WDK,Year,2015-2019,FAC_WEMAX,1.168174
28488,WDK,Year,2016,FAC_WEMAX,1.168849
28489,WDK,Year,2018,FAC_WEMAX,1.161396
28490,WDK,Year,2019,FAC_WEMAX,1.163974


In [57]:
dfFactors_Base = dfSeasonFactors[0:0] #just copy data structure, no row data

dfFactors_Base = dfFactors_Base.append(dfSeasonFactors              , ignore_index=True)
dfFactors_Base = dfFactors_Base.append(dfMaxMonthFactorMo           , ignore_index=True)
dfFactors_Base = dfFactors_Base.append(dfMaxMonthFactorMax          , ignore_index=True)
dfFactors_Base = dfFactors_Base.append(dfSeasonGroupDOWFactors_toCombine      , ignore_index=True)
dfFactors_Base = dfFactors_Base.append(dfSeasonGroupDOWFactorsMinMax_toCombine, ignore_index=True)

dfFactors_Base

,STATIONGROUP,SEASONGROUP,YEARGROUP,FACTORNAME,FACTOR
0,301,M01-Jan,2013,FAC_JAN,0.905472
1,301,M01-Jan,2013-2017,FAC_JAN,0.952858
2,301,M01-Jan,2014,FAC_JAN,0.985614
3,301,M01-Jan,2015,FAC_JAN,0.939451
4,301,M01-Jan,2015-2019,FAC_JAN,0.958303
...,...,...,...,...,...
316759,WDK,Year,2015-2019,FAC_WEMAX,1.168174
316760,WDK,Year,2016,FAC_WEMAX,1.168849
316761,WDK,Year,2018,FAC_WEMAX,1.161396
316762,WDK,Year,2019,FAC_WEMAX,1.163974


# Add Derived Station Groups

In [58]:
dfFactors_Derived = pd.DataFrame.merge(prmDerivedStationPortions,dfFactors_Base,left_on=sStationGrpCol + 'DERIVEDFROM',right_on=sStationGrpCol)
dfFactors_Derived['FACTOR_NEW'] = dfFactors_Derived['PORTION'] * dfFactors_Derived['FACTOR']
dfFactors_Derived = dfFactors_Derived.rename(columns={sStationGrpCol+'_x':sStationGrpCol})
dfFactors_Derived = dfFactors_Derived.groupby([sStationGrpCol,sSeasonGrpCol,sYearGrpCol,'FACTORNAME'],as_index=False).agg(FACTOR=('FACTOR_NEW','sum'),RECORDCNT=('FACTOR_NEW','size'))
display(dfFactors_Derived)
dfFactors_Derived = dfFactors_Derived[[sStationGrpCol,sSeasonGrpCol,sYearGrpCol,'FACTORNAME','FACTOR']]
display(dfFactors_Derived)

,STATIONGROUP,SEASONGROUP,YEARGROUP,FACTORNAME,FACTOR,RECORDCNT
0,SNB,M01-Jan,2013,FAC_FRI,1.092775,2
1,SNB,M01-Jan,2013,FAC_JAN,0.865805,2
2,SNB,M01-Jan,2013,FAC_MON,0.999166,2
3,SNB,M01-Jan,2013,FAC_SAT,0.965680,2
4,SNB,M01-Jan,2013,FAC_SUN,0.738281,2
...,...,...,...,...,...,...
11146,XX4,Year,All Years,FAC_WDAVG,1.145891,3
11147,XX4,Year,All Years,FAC_WEAVG,0.805478,3
11148,XX4,Year,All Years,FAC_WED,1.159972,3
11149,XX4,Year,All Years,FAC_WEMAX,1.160095,3


,STATIONGROUP,SEASONGROUP,YEARGROUP,FACTORNAME,FACTOR
0,SNB,M01-Jan,2013,FAC_FRI,1.092775
1,SNB,M01-Jan,2013,FAC_JAN,0.865805
2,SNB,M01-Jan,2013,FAC_MON,0.999166
3,SNB,M01-Jan,2013,FAC_SAT,0.965680
4,SNB,M01-Jan,2013,FAC_SUN,0.738281
...,...,...,...,...,...
11146,XX4,Year,All Years,FAC_WDAVG,1.145891
11147,XX4,Year,All Years,FAC_WEAVG,0.805478
11148,XX4,Year,All Years,FAC_WED,1.159972
11149,XX4,Year,All Years,FAC_WEMAX,1.160095


In [59]:
dfFactors = dfFactors_Base.append(dfFactors_Derived, ignore_index=True)
dfFactors

,STATIONGROUP,SEASONGROUP,YEARGROUP,FACTORNAME,FACTOR
0,301,M01-Jan,2013,FAC_JAN,0.905472
1,301,M01-Jan,2013-2017,FAC_JAN,0.952858
2,301,M01-Jan,2014,FAC_JAN,0.985614
3,301,M01-Jan,2015,FAC_JAN,0.939451
4,301,M01-Jan,2015-2019,FAC_JAN,0.958303
...,...,...,...,...,...
327910,XX4,Year,All Years,FAC_WDAVG,1.145891
327911,XX4,Year,All Years,FAC_WEAVG,0.805478
327912,XX4,Year,All Years,FAC_WED,1.159972
327913,XX4,Year,All Years,FAC_WEMAX,1.160095


In [60]:
#check to see if all stations are there
dfFactors[sStationGrpCol].unique()

array(['301', '302', '303', '304', '305', '306', '307', '308', '309',
       '310', '312', '313', '314', '315', '316', '317', '318', '319',
       '320', '321', '322', '323', '324', '325', '327', '329', '332',
       '333', '335', '340', '341', '348', '349', '350', '351', '353',
       '354', '355', '362', '363', '382', '400', '401', '402', '403',
       '404', '405', '406', '407', '408', '409', '411', '412', '414',
       '415', '416', '418', '420', '421', '424', '425', '427', '430',
       '431', '501', '502', '503', '504', '506', '507', '508', '509',
       '510', '511', '512', '513', '601', '602', '605', '606', '609',
       '611', '612', '613', '614', '615', '616', '617', '619', '620',
       '621', '622', '623', '624', '625', '626', '627', '628', '629',
       '630', '631', '632', '633', '634', '635', '636', '703', '704',
       '711', '712', '713', '714', 'ALL', 'SN0', 'SN1', 'SN2', 'SN3',
       'SN4', 'SN5', 'SN6', 'SN7', 'SN8', 'SN9', 'SNA', 'SND', 'SNE',
       'SNF', 'SNG',

In [61]:
dfFactors.to_csv(csv_factors,index=False)

# Create Web App Data

In [62]:
#export DOW factors for web app

dfDOWForWebApp = pd.DataFrame.merge(dfFactors,prmWebAppDOWGroups,left_on=('FACTORNAME'),right_on=(sDOWFieldName))
dfDOWForWebApp = dfDOWForWebApp[dfDOWForWebApp['DAYGROUP']==1]
dfDOWForWebApp = dfDOWForWebApp[[sStationGrpCol,sYearGrpCol,sSeasonGrpCol,'DAYNAME','FACTOR']]
dfDOWForWebApp = dfDOWForWebApp.rename(columns={sStationGrpCol:'SG',sYearGrpCol:'YG','SEASONGROUP':'MG','DAYNAME':'D','FACTOR':'F'})
dfDOWForWebApp['F'] = dfDOWForWebApp['F'].astype(str).str[:6]
#dfDOWForWebApp = dfDOWForWebApp[-dfDOWForWebApp.SG.str[0].str.isalpha()] #station group starts with numeric character
dfDOWForWebApp.to_json(json_dowfactors,orient='records')
dfDOWForWebApp

,SG,YG,MG,D,F
58990,301,2013,M01-Jan,Mon,0.9192
58991,301,2013-2017,M01-Jan,Mon,0.9501
58992,301,2014,M01-Jan,Mon,0.9411
58993,301,2015,M01-Jan,Mon,0.9310
58994,301,2015-2019,M01-Jan,Mon,0.9715
...,...,...,...,...,...
265450,XX4,2016,Year,Sun,0.5263
265451,XX4,2017,Year,Sun,0.5192
265452,XX4,2018,Year,Sun,0.5211
265453,XX4,2019,Year,Sun,0.5400


In [63]:
#export DOW factors for web app

dfDOWForWebApp_StationGroup = pd.DataFrame.merge(dfFactors,prmWebAppDOWGroups,left_on=('FACTORNAME'),right_on=(sDOWFieldName))
dfDOWForWebApp_StationGroup = dfDOWForWebApp_StationGroup[dfDOWForWebApp_StationGroup['DAYGROUP']==2]
dfDOWForWebApp_StationGroup = dfDOWForWebApp_StationGroup[[sStationGrpCol,sYearGrpCol,sSeasonGrpCol,'DAYNAME','FACTOR']]
dfDOWForWebApp_StationGroup = dfDOWForWebApp_StationGroup.rename(columns={sStationGrpCol:'SG',sYearGrpCol:'YG','SEASONGROUP':'MG','DAYNAME':'D','FACTOR':'F'})
dfDOWForWebApp_StationGroup['F'] = dfDOWForWebApp_StationGroup['F'].astype(str).str[:6]
dfDOWForWebApp_StationGroup = dfDOWForWebApp_StationGroup[dfDOWForWebApp_StationGroup.SG.str[0].str.isalpha()] #station group starts with alpha character
dfDOWForWebApp_StationGroup.to_json(json_wdfactors,orient='records')
dfDOWForWebApp_StationGroup

,SG,YG,MG,D,F
17510,ALL,2013,M01-Jan,W1-Mon-Thu,1.0829
17511,ALL,2013-2017,M01-Jan,W1-Mon-Thu,1.0715
17512,ALL,2014,M01-Jan,W1-Mon-Thu,1.0627
17513,ALL,2015,M01-Jan,W1-Mon-Thu,1.0682
17514,ALL,2015-2019,M01-Jan,W1-Mon-Thu,1.0726
...,...,...,...,...,...
294945,XX4,2016,Year,MaxWkEnd,1.1616
294946,XX4,2017,Year,MaxWkEnd,1.1613
294947,XX4,2018,Year,MaxWkEnd,1.1605
294948,XX4,2019,Year,MaxWkEnd,1.1505


In [64]:
dfDOWForWebApp_StationGroup

,SG,YG,MG,D,F
17510,ALL,2013,M01-Jan,W1-Mon-Thu,1.0829
17511,ALL,2013-2017,M01-Jan,W1-Mon-Thu,1.0715
17512,ALL,2014,M01-Jan,W1-Mon-Thu,1.0627
17513,ALL,2015,M01-Jan,W1-Mon-Thu,1.0682
17514,ALL,2015-2019,M01-Jan,W1-Mon-Thu,1.0726
...,...,...,...,...,...
294945,XX4,2016,Year,MaxWkEnd,1.1616
294946,XX4,2017,Year,MaxWkEnd,1.1613
294947,XX4,2018,Year,MaxWkEnd,1.1605
294948,XX4,2019,Year,MaxWkEnd,1.1505


In [65]:
display(prmStationGroups)
#only use if first character of STATIONGROUP is alpha not numeric
dfStationForWebApp = prmStationGroups[prmStationGroups.STATIONGROUP.str[0].str.isalpha()]
dfStationForWebApp = dfStationForWebApp.rename(columns={sStationGrpCol:'value','STATIONGROUPDESCRIPTION':'label'})
dfStationForWebApp

,STATIONGROUP,STATIONGROUPDESCRIPTION
0,301,"301: I 80 1 mile E of I 215 Int., Parleys Cany..."
1,302,302: I 15 12900 South M.P. 290.600 FC 02
2,303,"303: SR 91 1.5 miles N of SR 61, Webster Jct M..."
3,304,"304: SR 89 0.6 mile N of SR 30, Garden City MP..."
4,305,"305: SR 24 0.1 mile N of SR 118, Sigurd MP 8.0..."
...,...,...
242,WDG,WDG: Weekday Freeway I-84 East
243,WDH,WDH: Weekday Freeway I-84 West
244,WDI,WDI: Weekday Freeway Legacy
245,WDJ,WDJ: Weekday Freeway SR-201


,value,label
175,ALL,All: All Stations in Utah
176,SN0,SN0: Season Alpine Loop
177,SN1,SN1: Season Bear Lake
178,SN2,SN2: Season Big Cottonwood
179,SN3,SN3: Season Cedar Breaks
...,...,...
242,WDG,WDG: Weekday Freeway I-84 East
243,WDH,WDH: Weekday Freeway I-84 West
244,WDI,WDI: Weekday Freeway Legacy
245,WDJ,WDJ: Weekday Freeway SR-201


In [66]:
dfStationForWebApp.to_json(json_stationgroups,orient='records')

In [67]:
#year groups
dfYearGroups = prmYearGroups
dfYearGroups['value'] = dfYearGroups['YEARGROUP']
dfYearGroups['label'] = dfYearGroups['YEARGROUP']
dfYearGroups = dfYearGroups[['value','label']]
dfYearGroups.to_json(json_yeargroups,orient='records')
dfYearGroups

,value,label
0,2013,2013
1,2014,2014
2,2015,2015
3,2016,2016
4,2017,2017
5,2018,2018
6,2019,2019
7,2013-2017,2013-2017
8,2015-2019,2015-2019
9,All Years,All Years


In [68]:
#season factors
dfSeasonForWebApp = pd.DataFrame.merge(dfFactors,prmWebAppSeasonGroups,left_on=('FACTORNAME'),right_on=(sSGFieldName))
dfSeasonForWebApp = dfSeasonForWebApp[dfSeasonForWebApp['GROUP']==1]
dfSeasonForWebApp = dfSeasonForWebApp[[sStationGrpCol,sYearGrpCol,'NAME','FACTOR']]
dfSeasonForWebApp = dfSeasonForWebApp.rename(columns={sStationGrpCol:'SG',sYearGrpCol:'YG','NAME':'M','FACTOR':'F'})
dfSeasonForWebApp['F'] = dfSeasonForWebApp['F'].astype(str).str[:6]
dfSeasonForWebApp.to_json(json_monthfactors,orient='records')
dfSeasonForWebApp

,SG,YG,M,F
0,301,2013,Jan,0.9054
1,301,2013-2017,Jan,0.9528
2,301,2014,Jan,0.9856
3,301,2015,Jan,0.9394
4,301,2015-2019,Jan,0.9583
...,...,...,...,...
20815,XX4,2016,Dec,0.9663
20816,XX4,2017,Dec,0.9940
20817,XX4,2018,Dec,0.9844
20818,XX4,2019,Dec,0.9092


In [69]:
#export DOW factors for web app

dfSeasonForWebApp_StationGroup = pd.DataFrame.merge(dfFactors,prmWebAppSeasonGroups,left_on=('FACTORNAME'),right_on=(sSGFieldName))
dfSeasonForWebApp_StationGroup = dfSeasonForWebApp_StationGroup[dfSeasonForWebApp_StationGroup['GROUP']==2]
dfSeasonForWebApp_StationGroup = dfSeasonForWebApp_StationGroup[[sStationGrpCol,sYearGrpCol,'NAME','FACTOR']]
dfSeasonForWebApp_StationGroup = dfSeasonForWebApp_StationGroup.rename(columns={sStationGrpCol:'SG',sYearGrpCol:'YG','NAME':'M','FACTOR':'F'})
dfSeasonForWebApp_StationGroup['F'] = dfSeasonForWebApp_StationGroup['F'].astype(str).str[:6]
dfSeasonForWebApp_StationGroup = dfSeasonForWebApp_StationGroup[dfSeasonForWebApp_StationGroup.SG.str[0].str.isalpha()] #station group starts with alpha character
dfSeasonForWebApp_StationGroup.to_json(json_seasonfactors,orient='records')
dfSeasonForWebApp_StationGroup

,SG,YG,M,F
21850,ALL,2013,Winter,0.9141
21851,ALL,2013-2017,Winter,0.9298
21852,ALL,2014,Winter,0.9307
21853,ALL,2015,Winter,0.9380
21854,ALL,2015-2019,Winter,0.9339
...,...,...,...,...
27755,XX4,2016,Fall,1.0039
27756,XX4,2017,Fall,1.0035
27757,XX4,2018,Fall,1.0230
27758,XX4,2019,Fall,1.0252


In [70]:
# station groups

dfStationsFromStationGroups = prmStationGroups[-prmStationGroups.STATIONGROUP.str[0].str.isalpha()].copy()
dfStationsFromStationGroups = dfStationsFromStationGroups.rename(columns={'STATIONGROUP':'StationID','STATIONGROUPDESCRIPTION':'StationName'})
dfStationsFromStationGroups['StationID'] = dfStationsFromStationGroups['StationID'].astype('int64')
dfStationsFromStationGroups

#prmStationToStationGroups
dfStationGroupsNoSingleStations = prmStationToStationGroups[prmStationToStationGroups.STATIONGROUP.str[0].str.isalpha()].copy()
dfStationGroupsNoSingleStations = dfStationGroupsNoSingleStations.rename(columns={'STATION':'StationID','STATIONGROUP':'StationGroup'})
dfStationGroupsNoSingleStations

dfStationGroupsCCS = pd.DataFrame.merge(dfStationsFromStationGroups,dfStationGroupsNoSingleStations,on='StationID')
dfStationGroupsCCS = dfStationGroupsCCS[['StationGroup','StationID','StationName']]
dfStationGroupsCCS = dfStationGroupsCCS.sort_values(by=['StationGroup','StationID'])
dfStationGroupsCCS

,StationGroup,StationID,StationName
0,ALL,301,"301: I 80 1 mile E of I 215 Int., Parleys Cany..."
4,ALL,302,302: I 15 12900 South M.P. 290.600 FC 02
8,ALL,303,"303: SR 91 1.5 miles N of SR 61, Webster Jct M..."
12,ALL,304,"304: SR 89 0.6 mile N of SR 30, Garden City MP..."
17,ALL,305,"305: SR 24 0.1 mile N of SR 118, Sigurd MP 8.0..."
...,...,...,...
349,WDH,614,614: I 84 3.2 miles W of Bothwell Int. MP 36.6...
385,WDI,624,624: SR 67 Legacy Parkway MP 0.944 FC 12
389,WDI,625,625: SR 67 Legacy Parkway MP 11.418 FC 12
365,WDJ,619,"619: SR 201 6174 W, SLC MP 10.600 FC 12"
